# Imports :

In [7]:
import numpy as np
import pandas as pd
import os
import cv2
from shutil import copy2

from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, BatchNormalization, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("imported")

imported


## Constants :

In [8]:
path = 'images/'
img_size = (224, 224)

# Data preprocess :

In [3]:
data = pd.read_csv("metadata.csv")
data = data.dropna(subset=['finding'])
data.columns

Index(['patientid', 'offset', 'sex', 'age', 'finding', 'survival', 'intubated',
       'intubation_present', 'went_icu', 'in_icu', 'needed_supplemental_O2',
       'extubated', 'temperature', 'pO2_saturation', 'leukocyte_count',
       'neutrophil_count', 'lymphocyte_count', 'view', 'modality', 'date',
       'location', 'folder', 'filename', 'doi', 'url', 'license',
       'clinical_notes', 'other_notes', 'Unnamed: 28'],
      dtype='object')

## The columns that we're interested in are : finding, view, filename

In [9]:
data.head()

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0,M,65.0,COVID-19,Y,N,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3,M,65.0,COVID-19,Y,N,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5,M,65.0,COVID-19,Y,N,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6,M,65.0,COVID-19,Y,N,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0,F,52.0,COVID-19,NaN,N,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


## creating a specific folder for each class :

In [24]:
if not os.path.exists('classes'):
    os.makedirs('classes/positive')
    os.makedirs('classes/negative')

filenames = os.listdir(path)
for file in filenames:
    image = cv2.imread(path + file, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, img_size)
    img_data = data.where(data['filename'] == file).dropna(how='all')
    
    if(img_data['finding'].isna().all() or img_data['view'].values[0] == 'Axial'):
        continue
    
    img_class = img_data['finding'].values[0]
    repo = 'positive/' if (img_class == 'COVID-19') else 'negative/'
    cv2.imwrite('classes/' + repo + file, image)
    

# Creating the model :

In [26]:
inp = Input(shape=img_size + (1,))

X = Conv2D(32, (3, 3), (2, 2), padding='same', kernel_initializer='he_uniform')(inp)
X = BatchNormalization(momentum=0.2)(X)
X = LeakyReLU(alpha=0.3)(X)

X = Conv2D(64, (3, 3), (2, 2), padding='same', kernel_initializer='he_uniform')(X)
X = BatchNormalization(momentum=0.2)(X)
X = LeakyReLU(alpha=0.3)(X)

X = Conv2D(128, (3, 3), (2, 2), padding='same', kernel_initializer='he_uniform')(X)
X = BatchNormalization(momentum=0.2)(X)
X = LeakyReLU(alpha=0.3)(X)

X = Conv2D(256, (3, 3), (2, 2), padding='same', kernel_initializer='he_uniform')(X)
X = BatchNormalization(momentum=0.2)(X)
X = LeakyReLU(alpha=0.3)(X)

X = Conv2D(512, (3, 3), (2, 2), padding='same', kernel_initializer='he_uniform')(X)
X = BatchNormalization(momentum=0.2)(X)
X = LeakyReLU(alpha=0.3)(X)

X = Flatten()(X)
X = Dense(64, activation='relu', kernel_initializer='he_uniform')(X)
X = Dense(2, activation='softmax', name='output')(X)

model = Model(inp, X)
model.summary()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 64)        256       
_________

## Image data generator :

In [34]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

generator = datagen.flow_from_directory('classes/',
    class_mode='categorical', batch_size=32, target_size=(224, 224), color_mode="grayscale", shuffle=True, seed=42
)

Found 690 images belonging to 2 classes.


In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit_generator(generator=generator, steps_per_epoch=len(generator), 
                    validation_data=generator, validation_steps=len(generator),epochs=30)

Epoch 1/30
22/22 [==============================] - 19s 868ms/step - loss: 1.3010 - acc: 0.6435 - val_loss: 0.6201 - val_acc: 0.6710
Epoch 2/30
22/22 [==============================] - 4s 163ms/step - loss: 0.6062 - acc: 0.6623 - val_loss: 0.5672 - val_acc: 0.6710
Epoch 3/30
22/22 [==============================] - 4s 166ms/step - loss: 0.5833 - acc: 0.6754 - val_loss: 0.5445 - val_acc: 0.6928
Epoch 4/30
22/22 [==============================] - 4s 163ms/step - loss: 0.5401 - acc: 0.6971 - val_loss: 0.4939 - val_acc: 0.7101
Epoch 5/30
22/22 [==============================] - 4s 162ms/step - loss: 0.4487 - acc: 0.7870 - val_loss: 0.3797 - val_acc: 0.8261
Epoch 6/30
22/22 [==============================] - 4s 161ms/step - loss: 0.3898 - acc: 0.8203 - val_loss: 0.3211 - val_acc: 0.8609
Epoch 7/30
22/22 [==============================] - 4s 166ms/step - loss: 0.2823 - acc: 0.8884 - val_loss: 0.2032 - val_acc: 0.9406
Epoch 8/30
22/22 [==============================] - 4s 169ms/step - loss: 0